# Sub-Numpy
We will create our own implementation of a few functionalities supported by the NumPy
Library. We will call our implementation SNumPy (for Sub-NumPy). SNumPy will be the name of the
class you implement, and we will refer to it by the shorthand “snp” from here on

In [73]:
# Create SNumPy class that will hold our methods for data manipulation methods,  not using numpy
# Including snp.ones(Int), snp.zeros(Int), snp.reshape(array, (row, column)), snp.shape(array), snp.append(array1, array2)
# snp.get(array, (row, column)), snp.add(array1, array1), snp.subtract(array1, array1), snp.dotproduct(array1, array1)

class SNumPy:
    """
    SNumPy class for basic array manipulations.
    """

    @staticmethod
    def ones(n, m=None):
        """
        Return an array of ones in given shape (n,m).
        """
        if m == None:
            return [1 for i in range(n)]
        else:
            return [[1 for i in range(m)] for j in range(n)]


    @staticmethod
    def zeros(n, m=None):
        """
        Return an array of zeros in given shape (n,m).
        """
        if m == None:
            return [0 for i in range(n)]
        else:
            return [[0 for i in range(m)] for j in range(n)]

    @staticmethod
    def reshape(array, shape):
        """
        Return an array containing the same data with a new shape.
        """
        column, row = shape
        new_array = []
        for i in range(row):
            new_array.append(array[i*column:(i+1)*column])
        return new_array

    @staticmethod
    def shape(array):
        """
        Return the shape of an array.
        """
        # Check if it's a vector (1D array)
        if not array or not isinstance(array[0], list):
            return (len(array),)
        # Else, it's a matrix (2D array)
        else:
            return (len(array), len(array[0]))

    @staticmethod
    def append(array1, array2):
        """
        Return an array containing the same data with a new shape.
        """
        return array1 + array2

    @staticmethod
    def get(array, index):
        """
        Return the element at the given index.
        """
        column, row = index  # Unpack the index tuple
        return array[row][column]

    @staticmethod
    def add(array1, array2):
        """
        Return the element-wise sum of two arrays.
        """
        try:
            if SNumPy.shape(array1) != SNumPy.shape(array2):
                raise ValueError("Arrays must be the same size")
        except ValueError as e:
            print(e)
            return None

        # Using zip() will parrallelize the addition of the arrays
        return [
            [
                cell1 + cell2 
                for cell1, cell2 in zip(row1, row2)
            ] 
            for row1, row2 in zip(array1, array2)
        ]

    @staticmethod
    def subtract(array1, array2):
        """
        Return the element-wise difference of two arrays.
        """
        try:
            if SNumPy.shape(array1) != SNumPy.shape(array2):
                raise ValueError("Arrays must be the same size")
        except ValueError as e:
            print(e)
            return None
        
        # Using zip() will parrallelize the subtraction of the arrays
        return [
            [
                cell1 - cell2 
                for cell1, cell2 in zip(row1, row2)
            ] 
            for row1, row2 in zip(array1, array2)
        ]

    @staticmethod
    def dotproduct(array1, array2):
        """
        Check if its vectors or matrices, then return the dot product.
        """
        # Check if its vectors
        if len(SNumPy.shape(array1)) == 1 and len(SNumPy.shape(array2)) == 1:
            # Check if the vectors are the same size
            try:
                if SNumPy.shape(array1) != SNumPy.shape(array2):
                    raise ValueError("Vectors must be the same size")
            except ValueError as e:
                print(e)
                return None

            return sum([array1[i] * array2[i] for i in range(len(array1))])
        
        # Else do matrix multiplication
        else:
            # Check if the columns of array1 are the same size as the rows of array2
            try:
                if SNumPy.shape(array1)[1] != SNumPy.shape(array2)[0]:
                    raise ValueError(f"Array1's columns '{SNumPy.shape(array1)[1]}'  must be the same size as Array2's rows '{SNumPy.shape(array2)[0]}'")
            except ValueError as e:
                print(e)
                return None

            return [
                [
                    sum(
                        [array1[i][k] * array2[k][j] 
                        for k in range(len(array1[0]))]
                    ) 
                    for j in range(len(array2[0]))
                ] 
                for i in range(len(array1))
            ]
        
    @staticmethod
    def scalar_multiply(array, scalar):
        """
        Return the scalar product of an array.
        """
        return [
            [
                cell * scalar
                for cell in row
            ]
            for row in array
        ]
        
 

In [78]:
# Test cases for SNumPy class

snp = SNumPy()
print(snp.ones(5))
print(snp.zeros(5))
print(snp.reshape([1, 2, 3, 4, 5, 6], (2, 3)))
print(snp.shape([[1, 2, 3], [4, 5, 6]]))
print(snp.append([1, 2, 3], [4, 5, 6]))
print(snp.get([[1, 2, 3], [4, 5, 6]], (1, 1)))
print(snp.add([[1, 2, 3], [4, 5, 6]], [[1, 2, 3], [4, 5, 6]]))
print(snp.subtract([[1, 2, 3], [4, 5, 6]], [[1, 2, 3], [4, 5, 6]]))
print(snp.dotproduct([[1, 2, 3]], [[4, 5, 6], [1, 2, 3], [7, 8, 9]]))
print(snp.scalar_multiply([[1, 2, 3], [4, 5, 6]], 2))

[1, 1, 1, 1, 1]
[0, 0, 0, 0, 0]
[[1, 2], [3, 4], [5, 6]]
(2, 3)
[1, 2, 3, 4, 5, 6]
5
[[2, 4, 6], [8, 10, 12]]
[[0, 0, 0], [0, 0, 0]]
[[27, 33, 39]]
[[2, 4, 6], [8, 10, 12]]
